In [ ]:
from google.colab import files
files.upload()

In [ ]:
import os
os.makedirs('/root/.kaggle', exist_ok=True)
os.rename('kaggle.json', '/root/.kaggle/kaggle.json')

In [ ]:
!pip install kaggle

!kaggle competitions download -c titanic

In [ ]:
!unzip titanic.zip

In [ ]:
import pandas as pd
import numpy as np
import random as rnd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
!pwd
!ls

In [ ]:
train_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/test.csv')
combine = [train_df, test_df]
print(train_df.shape)
combine[0].shape

In [ ]:
print(train_df.columns.values)

In [ ]:
# 범주형 데이터(범주형: Survived, Sex, Embarked, 서열형(순서) : Pclass)
# 수치형 데이터(연속형 : Age, 이산형 : SibSp, Parch)

train_df.head()

In [ ]:
train_df.tail()

In [ ]:
# 결측값이 존재하는지 확인하는 방법은?
train_df.info()
print('-'*40)
test_df.info()

# 결측되지 않은 값, 데이터 타입(64타입 정수, objext(문자열), 64타입 부동소수)

In [ ]:
# 주식별자, Pclass, 이름, 성별, 가족관계(부모자녀/형제배우자), 티켓, 생존여부, 요금

train_df.describe()

In [ ]:
survived_percentiles = train_df['Survived'].quantile([.61, .62])
print(survived_percentiles)

parch_percentiles = train_df['Parch'].quantile([.75, .8])
print(parch_percentiles)

sibsp_percentiles = train_df['SibSp'].quantile([.68, .69])
print(sibsp_percentiles)

age_percentiles = train_df['Age'].quantile([ .1, .2, .3, .4, .5, .6, .7, .8, .9, .99])
print(age_percentiles)

In [ ]:
# 이름 : 전부 다름, 성별 : 두개, 객실 : 다중복(그룹화), 항구 : 세개, 티켓 : 22% 중복(왜?)
# O:Pandas/object 타입을 간략하게 나타내는 표현, top:최다 등장, freq:top개수, unique:유형수

train_df.describe(include=['O'])

In [ ]:
# 생존과의 상관관계 분석
# 중요한 거 : 나이, 어디서 탔는지
# 버려도 될 것 같은 거 : Cabin, Ticket, Pessengerid, Name
# 특성 생성 : family, Title, Age bands, Fare range
# 분류 : 생존 확률 높았을 것이다 - 여성, 어린이, 상류층 (일반적인 경험, 데이터)
# Analyze by Pivoting features

In [ ]:
train_df[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)
# 객실 등급: 1등석(배의 상층부, 구명보트와 가까움), 2등석, 3등석

In [ ]:
train_df[["Sex", "Survived"]].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
train_df[["SibSp", "Survived"]].groupby(['SibSp'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
train_df[["Parch", "Survived"]].groupby(['Parch'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
# 숫자형 특성 vs 목표 변수 상관관계
# 히스토그램 : 연속적인 숫자형 변수 분석 유용
# 관찰, 결정(워크플로우 단계에서 반영해야함) -> 결측값 해결, 나이 그룹화

In [ ]:
# sns.FacetGrid() : Seaborn 라이브러리의 기능, 열의 값에 따라 여러 개의 그래프를 나누어 시각화
# 생존 그룹/사망 그룹 별도의 그래프로 나타냄
g = sns.FacetGrid(train_df, col='Survived')
g.map(plt.hist, 'Age', bins=20)

In [ ]:
grid = sns.FacetGrid(train_df, col='Survived', row='Pclass', height=2.2, aspect=1.6)
grid.map(plt.hist, 'Age', alpha=.5, bins=20)
grid.add_legend();

In [ ]:
grid = sns.FacetGrid(train_df, row='Embarked', height=2.2, aspect=1.6)
grid.map(sns.pointplot, 'Pclass', 'Survived', 'Sex', palette='deep')
grid.add_legend()

# 숫자가 아닌 값들과 숫자값의 상관관계 분석
# 1. 높은 요금 지불한 승객 생존 2. 탑승한 항구와 생존 관계
# 요금을 범주화 해서 그룹화하자 (저, 중, 고)

In [ ]:
# grid = sns.FacetGrid(train_df, col='Embarked', hue='Survived', palette={0: 'k', 1: 'w'})
grid = sns.FacetGrid(train_df, row='Embarked', col='Survived', height=2.2, aspect=1.6)
grid.map(sns.barplot, 'Sex', 'Fare', alpha=.5, errorbar=None)
grid.add_legend()

In [ ]:
# 불필요한 특징 제거 : 객실 정보, 티켓 번호 -> 훈련, 테스트 둘다 적용

In [ ]:
print("Before", train_df.shape, test_df.shape, combine[0].shape, combine[1].shape)

train_df = train_df.drop(['Ticket', 'Cabin'], axis=1)
test_df = test_df.drop(['Ticket', 'Cabin'], axis=1)
combine = [train_df, test_df]

"After", train_df.shape, test_df.shape, combine[0].shape, combine[1].shape

In [ ]:
# Title과 Survived간 상관관계 분석 -> 이후 Name, PassengerId 특성 삭제
# 정규 표현식(RegEx) : \w+\. == Name 특성 내에서 점으로 끝나는 첫 번째 단어 찾기. expand=False == DataFrame 형식으로 데이터 반환
# 관찰 후 -> Master 호칭 평균 나이 5세 / Title, Age 생존률 약간 차이 존재 / 특정 호칭은 생존여부에 의미
# 따라서 Title 특성 유지하기로 함
# A-Za-Z : 대소문자를 구분하지 않고 모든 알파벳(A-Z, a-z)포함하는 문자 집합

In [ ]:
# dataset['추출할_컬럼'].str.extract('정규표현식')
for dataset in combine:
  dataset['Title'] = dataset.Name.str.extract('([A-Za-z]+)\.', expand=False)

pd.crosstab(train_df['Title'], train_df['Sex'])

In [ ]:
# 성별, 나이, 사회적 지위 암시용으로 이름 구분
# 어린 남성, 어린 여성, 미혼 여성, 기혼 여성, 이외

for dataset in combine:
  dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess', 'Capt', 'Col', \
                                               'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
  dataset['Title'] = dataset['Title'].replace(['Mlle', 'Ms'], 'Miss')
  dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

train_df[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

In [ ]:
grouped = train_df.groupby(['Survived', 'Title']).size().reset_index(name='Count')
print(grouped)

In [ ]:
title_mapping = {'Mr':1, 'Miss':2, 'Mrs':3, 'Master':4, 'Rare':5}
for dataset in combine:
  dataset['Title'] = dataset['Title'].map(title_mapping)
  dataset['Title'] = dataset['Title'].fillna(0)

train_df.head()

In [ ]:
train_df = train_df.drop(['Name', 'PassengerId'], axis=1)
test_df = test_df.drop(['Name'], axis=1)
combine = [train_df, test_df]
train_df.shape, test_df.shape

In [ ]:
train_df.columns

In [ ]:
test_df.columns

In [ ]:
# 이제 문자열을 포함하는 특성들을 수치형 값으로 변환할 수 있다.
# Sex 특성을 여자=1, 남자=0으로 분류된 Gender이라고 불리는 새로운 특성으로 변환하자

for dataset in combine:
  dataset['Sex'] = dataset['Sex'].map({'female':1, 'male':0}).astype(int)

print(dataset['Sex'].unique())
train_df.head()

In [ ]:
# 이제 결측값 처리
# 1. 평균, 표준편차 -> 노이즈
# 2. 상관관계가 있는 다른 특성 사용해 결측값 처리 (다른 특성들의 조건에 따라 결측값 달라짐)
# 3. Pclass, Gender 조합에 따라 Age 값으로 평균과 표준편차 사용 -> 노이즈

In [ ]:
grid = sns.FacetGrid(train_df, row='Pclass', col='Sex', height=2.2, aspect=1.6)
grid.map(plt.hist, 'Age', alpha=.5, bins=20)
grid.add_legend()

# 3등급 칸에서 남자 생존 비율이 월등히 높다

In [ ]:
guess_ages = np.zeros((2, 3))
guess_ages

In [ ]:
for dataset in combine:
  for i in range(0, 2):
    for j in range(0, 3):
      guess_df = dataset[(dataset['Sex']==i) & (dataset['Pclass']==j+1)]['Age'].dropna()

      age_guess = guess_df.median()

      guess_ages[i, j] = int(age_guess/0.5 + 0.5)*0.5

  for i in range(0, 2):
    for j in range(0, 3):
      dataset.loc[(dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1), \
                  'Age'] = guess_ages[i, j]

  dataset['Age'] = dataset['Age'].astype(int)

train_df.head()